In [ ]:
# %%
import os
import shutil
import json
from pathlib import Path
from datetime import date

# === CONFIGURATION ===
SOURCE_ROOT = Path("/home/debi/jaime/repos/MR-EyeTrack/results")
BIDS_ROOT = SOURCE_ROOT / "bids/derivatives/reconstructions"

# Reconstruction types
RECONS = {
    "steva": {
        # Full (no binning)
        "woBinning": "T1_LIBRE_woBinning/output/th8/x_steva_nIter_20_delta_1.000.nii.gz",
        # Motion-resolved (pattern under clean*/th8/)
        "pattern": "x_steva_regionidx_*_nIter_20_delta_1.000.nii.gz",
        "subfolder": "th8",
        "description": "L2-norm regularized reconstruction using STEVA algorithm"
    },
    "mathilda": {
        # Full (no binning)
        "woBinning": "T1_LIBRE_woBinning/output/x0/x0.nii.gz",
        # Motion-resolved (pattern under clean*/x0/)
        "pattern": "x0_regionidx*.nii.gz",
        "subfolder": "x0",
        "description": "Non-regularized reconstruction using MATHILDA algorithm"
    }
}

# Gaze direction map
GAZE_MAP = {"0": "up", "1": "down", "2": "left", "3": "right"}

# Undersampling map
UNDERSAMPLING_MAP = {
    "clean": "full",
    "clean_0.5": "50p",
    "clean_0.75": "75p",
    "clean_0.95": "95p"
}

# === MAIN SCRIPT ===
for sub_dir in sorted(SOURCE_ROOT.glob("Sub*")):
    subj_id = sub_dir.name.replace("Sub", "sub-")
    print(f"\nProcessing {subj_id}...")

    anat_dir = BIDS_ROOT / subj_id / "anat"
    anat_dir.mkdir(parents=True, exist_ok=True)

    for recon_type, recon_info in RECONS.items():

        # === 1️⃣ Full reconstruction (no binning) ===
        full_recon_path = sub_dir / recon_info["woBinning"]
        if full_recon_path.exists():
            bids_name = f"{subj_id}_rec-{recon_type}full_T1w.nii.gz"
            dest_path = anat_dir / bids_name
            shutil.copy(full_recon_path, dest_path)

            meta = {
                "ReconstructionDescription": f"{recon_info['description']} (full reconstruction, no motion binning)",
                "ReconstructionAlgorithm": recon_type,
                "FullReconstruction": True,
                "ReconstructionType": "static",
                "SkullStripped": False
            }

            json_path = dest_path.with_suffix("").with_suffix(".json")
            with open(json_path, "w") as f:
                json.dump(meta, f, indent=4)
        else:
            print(f"⚠️ Full {recon_type} reconstruction not found for {subj_id}")

        # === 2️⃣ Motion-resolved reconstructions ===
        binning_root = sub_dir / "T1_LIBRE_Binning" / "output"

        for clean_folder, undersample_label in UNDERSAMPLING_MAP.items():
            folder = binning_root / clean_folder / recon_info["subfolder"]
            if not folder.exists():
                continue

            for nii in folder.glob(recon_info["pattern"]):
                # Extract region index → gaze direction
                region_idx = None
                for possible_sep in ["regionidx_", "regionidx"]:
                    if possible_sep in nii.name:
                        region_idx = nii.name.split(possible_sep)[1].split("_")[0].replace(".nii.gz", "").replace(".nii", "")
                        break

                gaze = GAZE_MAP.get(region_idx)
                if gaze is None:
                    continue

                bids_name = f"{subj_id}_acq-{gaze}_rec-{recon_type}{undersample_label}_T1w.nii.gz"
                dest_path = anat_dir / bids_name
                shutil.copy(nii, dest_path)

                if dest_path.exists():
                    meta = {
                        "ReconstructionDescription": (
                            f"{recon_info['description']} for {gaze} gaze, "
                            f"{'non-undersampled' if undersample_label=='full' else undersample_label + ' undersampling'}"
                        ),
                        "ReconstructionAlgorithm": recon_type,
                        "GazeDirection": gaze,
                        "UndersamplingRate": (
                            None if undersample_label == "full"
                            else float(undersample_label.replace('p', '')) / 100.0
                        ),
                        "ReconstructionType": "motion-resolved",
                        "FullReconstruction": undersample_label == "full",
                        "SkullStripped": False
                    }

                    json_path = dest_path.with_suffix("").with_suffix(".json")
                    with open(json_path, "w") as f:
                        json.dump(meta, f, indent=4)

print("\n✅ BIDS reconstruction dataset created under:", BIDS_ROOT)

# === 3️⃣ Create dataset_description.json ===
dataset_description = {
    "Name": "MR-EyeTrack Reconstructed MRI Dataset",
    "BIDSVersion": "1.9.0",
    "DatasetType": "derivative",
    "GeneratedBy": [
        {
            "Name": "STEVA",
            "Version": "1.0",
            "Description": "L2-norm regularized reconstruction algorithm"
        },
        {
            "Name": "MATHILDA",
            "Version": "1.0",
            "Description": "Non-regularized reconstruction algorithm"
        }
    ],
    "Acknowledgements": "Generated as part of the MR-EyeTrack project.",
    "HowToAcknowledge": "Please cite the MR-EyeTrack project when using this data.",
    "Date": str(date.today())
}

with open(BIDS_ROOT / "dataset_description.json", "w") as f:
    json.dump(dataset_description, f, indent=4)

print("\n📄 dataset_description.json created successfully at:", BIDS_ROOT)



Processing sub-001...

Processing sub-002...

Processing sub-003...

✅ BIDS reconstruction dataset created under: /home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions

📄 dataset_description.json created successfully at: /home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions
